In [39]:
"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv'] 
"""
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# os.listdir('./data_down')
pd.set_option("display.max_rows", 15, "display.max_columns", None, 'display.float_format', lambda x: '%.3f' % x)
# pd.set_option()


In [42]:
# read data
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
# df_esti = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
# df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
# df_bill = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
# df_bcaw = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv') 
# df_bill_type = pd.read_csv('./data_down/' + 'dim_dsc_billing_type_info.csv', sep = '\001') 
"""
pic part data
"""
sap_fina = pd.read_csv('./data_down/' + 'smart_hr_pl_0l.csv', sep = '\001') 
sap_fina['mm'] = [str(i).zfill(2) for i in sap_fina['mm']]
sap_fina['yrmon'] = sap_fina[['yy', 'mm']].astype(str).apply(lambda x: ''.join(x), axis = 1)
# levin_table = pd.read_excel('./data_down/' + 'OU_Segementation2.xlsx', sheet_name = 2) 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+\.{1}', '', i) for i in list(df.columns)]

for i in [df_oppa, df_bill,df_bill_type, df_esti,  df_cost,]:
    try:
        colname_modi(i)
    except:
        pass
del i
# rel = rel.drop(['SAP Customer', 'Finance Remark'], axis=1)

rel = rel[~rel['SAP Cost Center'].str.match('中|国')] 
rel.columns = ['cc_in_ou', 'sap_cust', 'bca', 'account_name_en']
rel['five_cc'] = rel['cc_in_ou'].str.slice(0,5)
rel2 = rel[['cc_in_ou', 'sap_cust', 'bca', 'five_cc']].merge(
    rel[['account_name_en', 'five_cc']].query("account_name_en != '(blank)'"
    ), on = 'five_cc', how = 'left').drop_duplicates()
rel2 = rel2.sort_values(['account_name_en', 'bca']).drop_duplicates(subset = 'cc_in_ou')

In [510]:
"""
my functions.
to customer level. functions set up.
"""
def data_clean(df, substr_year, **kwargs):
    """
    where u can set for month/year level.
    """
    df = df[~df['account_name_en'].isna()]
    df[substr_year] = df[substr_year].astype(str).str.slice(0,4) 
    df = df.groupby(['account_name_en', 'sap_cust', substr_year]).agg({
        **kwargs
    })
    df = df.rename({substr_year: 'month'}, axis = 1)
    return df 

def to_cust_level(df,  substr_year, **kwargs):
    """
    (df,  substr_year, **kwargs) 必须要有ou_code or cost_center!! 
    merge all to cust level \
        to_cust_level(df,  substr_year, **kwargs):
    """
    try:
        df1 = df.merge(rel2, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df1 = df.merge(rel2, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
    return data_clean(df1, substr_year,  **kwargs)

    
"""
opportunity select cols.
"""

def oppo_mas(op):
    """
    opportunity select cols.
    """
    columns = [
        'opportunity_number', 'account_name_en', 'account_name_cn', 
        'actual_close_date', 'createdon', 'expected_golive_date', 
        'annual_average_gross_profit','annual_average_revenue', 'bca_ref_number', 'bg', 'commission', 
        'contract_end_date', 'contract_term', 
        'milestone', 'opportunity_type', 'product1', 'product2','product3','product4', 'sector', 'territory',
        'contract_value', 'salesperson1', 'salesperson2', 'salesperson3', 'salesperson4'
        ]
    op = op[columns]
    
    def mon_mani(op, col):
        """钱多点"""
        op = op[op[col].astype(str).str.match('(\d+\.)').fillna(False)]
        op[col] = op[col].astype(float) * 1000    
        return op  
    for i in ['contract_value', 'annual_average_revenue', 'annual_average_gross_profit']:
        op = mon_mani(op, i)
    
    def op_cals(sap):
        """
        对op 内的财务数据进行sap相似处理.
        """
        sap['cogs'] = sap['annual_average_revenue'] - sap['annual_average_gross_profit']
        sap['gp_margin'] = sap['annual_average_gross_profit'] / sap['annual_average_revenue'] 
        sap['gp_margin'].replace([-np.inf, np.inf],  0, inplace = True)
        return sap
    op = op_cals(op)

    def time_mani(op, col):
        """
        时间格式化.
        """
        op[col] = op[col].str.slice(0,10).fillna(pd.NaT) 
        op[col] = pd.to_datetime(op[col])
        return op

    for i in ['actual_close_date', 'createdon', 'expected_golive_date', 'contract_end_date']:
        op = time_mani(op, i)

    
    op = op[~op['account_name_en'].str.contains('Freight Forwarding ').fillna(False)]
    op = op[op['account_name_en'] != 'Freight Forwarding Customer']
    # 删除同一批次导入的数据. 第一次.
    op['time_var'] = op['actual_close_date'] - op['createdon'] 
    op['time_var'] = op['time_var'].astype(str).str.extract('(\d+)').fillna(0).astype(int)

    return op
    

In [4]:
# """
# ~billing~
# bill detail table now having 50+ bms_types. which is unacceptalble/
# """
# def data_load_bms():
#     df = df_bill.copy()
#     df['month'] = df['bms_bill_end_date'].astype(str).str.slice(0,6) 
#     df = df.merge(df_bill_type, left_on = 'bms_fee_type', right_on = 'billing_fee_type_code', how = 'left')
#     bms = df.pivot_table(index= ['cost_center', 'month'],\
#         columns= 'category',
#         values='bms_amount',).reset_index().fillna(0)
#     return bms
# bms = data_load_bms()
# bms['yr'] = bms['month'].str.slice(0,4)
# bms_cus_yr = to_cust_level(df = bms, substr_year = 'month' ,\
#     cost_center = set, 
#     人力 = 'sum',
#     场地 = 'sum',
#     增值 = 'sum',
#     报关代理 = 'sum',
#     操作 = 'sum',
#     短驳运输 = 'sum',
#     索赔及返利 = 'sum',
#     设备使用 = 'sum' ).reset_index()
# bms_cus_yr.head(2)

In [454]:
"""
~sap~
"""
def load_data_sap():
    df = sap_fina.copy()
    df = df.rename({'prctr': 'cost_center'}, axis =1 )
    return df

sap_fina = load_data_sap()

sap = to_cust_level(sap_fina, 'yrmon', \
    cost_center = set, 
    income_amt = sum,
    gross_profit_amt = sum,
    human_cost_amt = sum, 
    service_outsourcing_amt = sum,
    labor_service_outsourcing_amt = sum
 
    ).reset_index()
    
sap = sap.rename({'yrmon':'mont_code'}, axis =1 )
sap['lb_cost'] = sap.iloc[:, -3:].sum(axis = 1)
sap.head(3)

,account_name_en,sap_cust,mont_code,cost_center,income_amt,gross_profit_amt,human_cost_amt,service_outsourcing_amt,labor_service_outsourcing_amt,lb_cost
0,"APEX Cargo Service Co., Ltd (ACS)",Freight Forwarding EI,2021,{EIXCNBGGFF},211150379.530,755858.640,218740.380,609.620,0.000,219350.000
1,APPLE,APPLE,2019,"{APPLESZWHW, APPLESHTRT, APPLESZFTS, APPLECDSB...",243470368.240,29580219.020,13782992.040,304678.730,848434.630,14936105.400
2,APPLE,APPLE,2020,"{APPLESZFTH, APPLESHTRH, APPLESZWHW, APPLESHTR...",230427929.000,26136167.910,14344433.470,230334.870,1267616.240,15842384.580


### finish load
---


> some opportunity analysis

In [392]:
# op = oppo_mas(df_oppa)
# op['mont_code'] = op['expected_golive_date'].astype(str).str.slice(0,7).str.replace('-', '')
# # op.head(2)
# op['prodct'] = op[['product1','product2', 'product3', 'product4']].astype(str).apply(lambda x: ','.join(x), axis = 1)
# op['prodct'] = op['prodct'].str.replace(',nan', '')
# op = op.drop(['product1','product2', 'product3', 'product4'], axis = 1)
# op['commission'] = op['commission'].replace('TRUE', 1).replace('FALSE', 0)
# # # from sklearn.preprocessing import MinMaxScaler
# # # scaler = MinMaxScaler() 
# # plt_cs = op[op['milestone'] == 'Contract Signed']
# # def bubble_plt(plot_data, color):
# #     fig = px.scatter(
# #         data_frame= plot_data, x = 'annual_average_revenue', y= 'time_var', hover_data=['account_name_en'],
# #         size = 'annual_average_gross_profit', color = color, title='  ')  
# #     # fig.add_vline(x = vline, line_width=3, line_dash="dash", line_color="black", opacity = .4)
# #     return fig
# # # bubble_plt(plt_cs,   'sector')
# # op2

---
# opdata
"""manipulation"""

In [515]:
import datetime
import calendar
from tqdm import tqdm
op = oppo_mas(df_oppa)
op['mont_code'] = op['expected_golive_date'].astype(str).str.slice(0,7).str.replace('-', '')
# op.head(2)
op['prodct'] = op[['product1','product2', 'product3', 'product4']].astype(str).apply(lambda x: ','.join(x), axis = 1)
op['sales'] = op[['salesperson1','salesperson2', 'salesperson3', 'salesperson4']].astype(str).apply(lambda x: ','.join(x), axis = 1)
op['prodct'] = op['prodct'].str.replace(',nan', '')
op['sales'] = op['sales'].str.replace(',nan', '')

op = op.drop(['product1','product2', 'product3', 'product4'], axis = 1)
op = op.drop(['salesperson1','salesperson2', 'salesperson3', 'salesperson4'], axis = 1)

op['commission'] = op['commission'].replace('TRUE', 1).replace('FALSE', 0)

In [518]:
op2 = op[op['milestone'] == 'Contract Signed']
"""
以月份为组的sum? 这里合理吗?
"""
op2 = op2.groupby(['account_name_en', 'account_name_cn', 'bg', 'mont_code', 'opportunity_number']).agg({
    'commission':sum,
    'annual_average_gross_profit' :sum, 
    'annual_average_revenue' :sum, 
    'contract_term' :sum, 
    'contract_value' :sum, 
    'prodct': set,
    'sales':set
}).reset_index()
op2['mon_to_yrend'] = (op2['mont_code'].str.slice(0,4) + '12').astype(int) - op2['mont_code'].astype(int)
monlist = [
    'annual_average_gross_profit',
    'annual_average_revenue',
    # 'contract_term',
    'contract_value',] 

op2['mont_code'] = op2['mont_code']+'01'


def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day).strftime('%Y%m')
# add_months(pd.to_datetime(op2['mont_code'])[0] ,1) 
q = []
for i, j in zip(pd.to_datetime(op2['mont_code']), op2['contract_term'].astype(int)):
    q.append( add_months(i, j))
op2['end_mont_code'] = q
op2['mon_to_yrend_future'] = (
    op2['end_mont_code'].astype(str).str.slice(0,4) + '12').astype(int) - op2['end_mont_code'].astype(int)


In [519]:
op2['dupli'] = op2['end_mont_code'].str.slice(0,4).astype(int) - op2['mont_code'].str.slice(0,4).astype(int)
tst = pd.DataFrame()
for i, j in tqdm(zip(range(op2.shape[0]), op2['dupli'])):
    """
    复制第一条数据, 产生每年年化收益.
    """
    try:
        tst = tst.append(pd.concat([pd.DataFrame(op2.iloc[i, :]).T] * int(j)))
    except:
        pass


for i in tqdm(monlist):
    """
    第一条数据已经转化为实际年收入
    """
    name = i
    op2[name] = op2[i].astype(float) / 12 * op2['mon_to_yrend']

993it [00:07, 134.74it/s]
100%|██████████| 3/3 [00:00<00:00, 250.03it/s]


In [520]:
"""
除掉第一年数据.
"""
tst['row_num'] = tst.groupby(['account_name_en', 'mont_code','annual_average_gross_profit']).cumcount() + 1
tst['mont_code'] = tst['mont_code'].str.slice(0,4).astype(int) + tst['row_num'].astype(int)
op2['mont_code'] = op2['mont_code'].str.slice(0,4).astype(int)
"""
中间年
"""
mid_yrs = tst.groupby([
    'account_name_en', 'contract_term', 'annual_average_gross_profit'], as_index=False
    ).apply(lambda x: x.iloc[:-1])

"""
结束年
"""
end_yrs = tst.groupby([
    'account_name_en', 'contract_term', 'annual_average_gross_profit'], as_index=False
    ).apply(lambda x: x.iloc[-1])

for i in tqdm(monlist):
    """
    结束年数据已经转化为实际年收入
    """
    name = i
    end_yrs[name] = end_yrs[i].astype(float) / 12 * (12 - end_yrs['mon_to_yrend'])

100%|██████████| 3/3 [00:00<00:00, 416.34it/s]


In [560]:
"""
concat all yrs to one
"""
op_all = pd.concat([op2, mid_yrs, end_yrs]).sort_values(
    ['account_name_en', 'mont_code']
    ).reset_index(

    ).drop_duplicates(subset = monlist + ['mont_code'])
op_all = op_all.drop('index', axis = 1)

In [523]:
# duplicated might caused by the rows who are first and last at the same time.? 
# not sure about it , but never mind since the value of these amount is short.
# op_all.head(4)

In [561]:
op_all['prodct'] =[' '.join(i) for i in op_all['prodct']]
op_all['sales'] =[' '.join(i) for i in op_all['sales']]

op_all = op_all.groupby(['account_name_en', 'account_name_cn', 'bg', 'mont_code']).agg(
    {
    'commission':sum,
    'annual_average_gross_profit' :sum, 
    'annual_average_revenue' :sum, 
    'contract_term' :'mean', 
    'contract_value' :sum, 
    'prodct': set  ,
    'sales': set,
    'opportunity_number': [set, 'count'],
    }
).reset_index()
op_all[('prodct', 'set')] = [';'.join(i) for i in op_all[('prodct', 'set')]]
op_all[('sales', 'set')] = [';'.join(i) for i in op_all[('sales', 'set')]]

In [563]:
op_all.columns = op_all.columns.droplevel(1)
list = op_all.columns.values
list[-1] = 'opportunity_number_num'
op_all.columns = list
del list
# del df_bill, df_bill_type, df_esti, i, j
# del q, op, op2, tst, mid_yrs, end_yrs
# del tst2

`===========================`
<br>
`merge sap`

In [457]:
"""
多行op 匹配单行的sap  
时间节点要用 actual go live 的yr. 对应当月的sap财务状况.
"""
sap['mont_code'] = sap['mont_code'].astype(int)
sap['cost_center'] = [','.join(i) for i in sap['cost_center']]

sap = sap.groupby(['account_name_en', 'sap_cust', 'mont_code']).agg(
    {
        'cost_center': set, 
        'income_amt': sum,
        'gross_profit_amt': sum, 
        'human_cost_amt': sum, 
        'service_outsourcing_amt': sum,
        'labor_service_outsourcing_amt': sum, 
        'lb_cost': sum
    }
).reset_index()

sap['cost_center'] = [','.join(i) for i in sap['cost_center']]


In [433]:
from fyenn_class import pd_loaddata

In [461]:
cc = pd_loaddata.pd_excel('./data_down/DSC成本中心_Complete', 0)
cc = cc.iloc[1:, [4,12]]
cc.columns = ['bg', 'cc_mapper']
cc['bg'] = cc['bg'].str.extract('(\w+$)')
sap['cc_mapper'] = sap['cost_center'].str.slice(0,10)
sap = sap.merge(cc, on = 'cc_mapper', how = 'inner') 

csv_read


In [460]:
###

In [564]:

# merge prepared
op_sap = op_all.merge(
    sap, on = ['account_name_en', 'mont_code', 'bg'], how = 'right'
    ).sort_values(['account_name_en', 'mont_code']);op_sap.shape
op_sap['gp_est_over_real'] = op_sap['annual_average_gross_profit'] / op_sap['gross_profit_amt']
op_sap['rev_est_over_real'] = op_sap['annual_average_revenue'] / op_sap['income_amt']

(719, 22)

In [568]:
# op_sap.to_pickle('./data_up/op_sap.p')
# op_sap.head(4)

In [573]:
op_all

,account_name_en,account_name_cn,bg,mont_code,commission,annual_average_gross_profit,annual_average_revenue,contract_term,contract_value,prodct,sales,opportunity_number,opportunity_number_num
0,ADIDAS,阿迪达斯,HK,2015,0,297000.000,3368000.000,36.000,10104000.000,"Warehousing,Warehousing",AlvinSC Chan (SF-DHL HK),{Z004273},1
1,ADIDAS,阿迪达斯,HK,2016,0,1188000.000,13472000.000,36.000,40416000.000,"Warehousing,Warehousing",AlvinSC Chan (SF-DHL HK),{Z004273},1
2,ADIDAS,阿迪达斯,HK,2017,0,1188000.000,13472000.000,36.000,40416000.000,"Warehousing,Warehousing",AlvinSC Chan (SF-DHL HK),{Z004273},1
3,ADIDAS,阿迪达斯,HK,2018,0,1143138.333,13040288.333,30.000,36184576.667,"Transportation;Warehousing,Warehousing",AlvinSC Chan (SF-DHL HK);nan,"{Z004481, Z004273}",2
4,ADIDAS,阿迪达斯,HK,2019,0,2012101.667,26201305.000,16.000,40650460.000,Transportation;Warehousing,StevenMT Wong (SF-DHL HK);CarmenSM Lo (SF-DHL ...,"{Z004481, Z004441, Z005460}",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1613,桂林深科技有限公司,桂林深科技有限公司,TSC,2021,1,26241.667,97115.000,3.000,45673.333,Warehousing,"潘荣,Xiaogang Yang (SF-DHL CN);潘荣,谢姝贤","{Z009647, Z010078}",2
1614,桂林深科技有限公司,桂林深科技有限公司,TSC,2022,0,40791.667,134241.667,3.000,140533.333,Warehousing,"潘荣,Xiaogang Yang (SF-DHL CN)",{Z010078},1
1615,翰林汇信息产业股份有限公司,翰林汇信息产业股份有限公司,CR,2021,0,10416.667,133750.000,12.000,133750.000,Other service,Hawkins He2 (SF-DHL CN),{Z009910},1
1616,翰林汇信息产业股份有限公司,翰林汇信息产业股份有限公司,CR,2022,0,14583.333,187250.000,12.000,187250.000,Other service,Hawkins He2 (SF-DHL CN),{Z009910},1


In [571]:
op_sap[op_sap['prodct'] != 'Freight Forwarding']

,account_name_en,account_name_cn,bg,mont_code,commission,annual_average_gross_profit,annual_average_revenue,contract_term,contract_value,prodct,sales,opportunity_number,opportunity_number_num,sap_cust,cost_center,income_amt,gross_profit_amt,human_cost_amt,service_outsourcing_amt,labor_service_outsourcing_amt,lb_cost,cc_mapper,gp_est_over_real,rev_est_over_real
1,APPLE,苹果,CM,2019,1.000,307930.000,2400000.000,36.000,7200000.000,Warehousing,Vivien He (SF-DHL CN),{Z005265},1.000,APPLE,"APPLESZWHW,APPLESHTRT,APPLESZFTS,APPLECDSBS,AP...",243470368.240,29580219.020,13782992.040,304678.730,848434.630,14936105.400,APPLESZWHW,0.010,0.010
2,APPLE,苹果,CM,2020,1.000,307930.000,2400000.000,36.000,7200000.000,Warehousing,Vivien He (SF-DHL CN),{Z005265},1.000,APPLE,"APPLESZFTH,APPLESHTRH,APPLESZWHW,APPLESHTRT,AP...",230427929.000,26136167.910,14344433.470,230334.870,1267616.240,15842384.580,APPLESZFTH,0.012,0.010
3,APPLE,苹果,CM,2021,1.000,307930.000,2400000.000,36.000,7200000.000,Warehousing,Vivien He (SF-DHL CN),{Z005265},1.000,APPLE,"APPLESZFTH,APPLESZWHW,APPLESHTRT,APPLESZFTS,AP...",269340827.350,29934238.830,12697653.060,248288.100,1426658.860,14372600.020,APPLESZFTH,0.010,0.009
4,APPLIED MATERIAL,NaN,TSC,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLIED MATERIAL,"AMATXSHKQS,AMATXWXIHS,AMATXHFTRT,AMATXXMXAS,AM...",181267688.410,19678995.810,16181380.400,96419.150,1626756.040,17904555.590,AMATXSHKQS,NaN,NaN
5,APPLIED MATERIAL,NaN,TSC,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLIED MATERIAL,"AMATXSHKQS,AMATXWXIHS,AMATXHFTRT,AMATXXMXAS,AM...",164716414.460,19662731.820,10134132.330,57188.390,943381.400,11134702.120,AMATXSHKQS,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,huayi life (Beijing) e-commerce,NaN,EI,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huayi life,HUAYIKMTRT,0.000,96623.000,0.000,0.000,0.000,0.000,HUAYIKMTRT,NaN,NaN
714,上海顺啸丰运输有限公司,上海顺啸丰运输有限公司,CR,2021,0.000,138750.000,5948270.000,12.000,5948270.000,Transportation,Hawkins He2 (SF-DHL CN),{Z009784},1.000,ECCO,SFECCSHTRD,7011367.180,196438.690,0.000,0.000,0.000,0.000,SFECCSHTRD,0.706,0.848
715,中都（株洲）物流有限公司,NaN,Auto,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCL,CCLXXJMTRD,3100652.330,167214.350,0.000,0.000,0.000,0.000,CCLXXJMTRD,NaN,NaN
716,北京卫宠健科技有限公司,北京卫宠健科技有限公司,TSC,2021,1.000,26530.000,354595.000,36.000,1063785.000,"Warehousing,Transportation","Jing Luo (SF-DHL CN),Yong Hua (SF-DHL CN)",{Z009590},1.000,Wei Chong Jian,WEICJBJXWS,359764.670,61472.940,39252.170,32260.810,0.000,71512.980,WEICJBJXWS,0.432,0.986


In [469]:
# def fillna_(col, op_sap):
#     name = 'prd_' + col
#     op_sap[name] = op_sap.groupby('account_name_en')[col].cumsum()
#     op_sap[name]  = op_sap .groupby('account_name_en')[name].fillna(method = 'bfill').fillna(method = 'ffill')
#     return op_sap
# for i in ['contract_value', 'annual_average_revenue', 'annual_average_gross_profit']:
#         op_sap = fillna_(i, op_sap)
plt2 = op_sap.groupby([''])

In [470]:
plt = plt2[plt2['account_name_en'].str.contains('APEX')]

# plt =op_sap[op_sap['mont_code'] < 202106]
# plt[~plt['annual_average_gross_profit'].isna()]
fig = go.Figure()
fig.add_trace( go.Line(  x = plt['mont_code'], y = plt['prd_annual_average_gross_profit']/12  , name = 'prd_annual_average_gross_profit'))
fig.add_trace( go.Line(  x = plt['mont_code'], y = plt['gross_profit_amt']  , name = 'gross_profit_amt'))
fig.add_trace( go.Line(  x = plt['mont_code'], y = plt['tt_bms']  , name = 'bms'))
# fig.add_trace( go.Line(  x = plt['mont_code'], y = plt['prd_contract_value']/12  ))
# fig.add_trace( go.Line(  x = plt['mont_code'], y = plt['prd_annual_average_revenue']/12  ))
# fig.add_trace( go.Line(  x = plt['mont_code'], y = plt['prd_annual_average_gross_profit']/12  ))


In [ ]:
# from fbprophet import Prophet
# # df = x[['CREATION_DATE_TIME_STAMP', 'qty']]
# df_naiv = x_train 
# df_naiv = df_naiv.rename({'date1' : 'ds'}, axis = 1)
# df_naiv_test = x_test
# df_naiv_test = df_naiv_test.rename({'date1' : 'ds'}, axis = 1)
# m = Prophet()
# [m.add_regressor(i) for i in cols]

# m.fit(df_naiv)

# # from sklearn.metrics import mean_absolute_percentage_error

# # # m.predict(x_test)

In [ ]:
# op = op[~(op['actual_close_date'] - op['createdon'] < datetime.timedelta(days = 0))] 
sap['max_mont'] = sap.groupby('account_name_en')['mont_code'].transform( 'max' )

In [ ]:
sap[sap['max_mont'] != '202110']